In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

In [11]:
import glob

# Directory where augmented images are stored
train_data_dir = "data-aug"

input_width, input_height = 300, 437
input_channels = 3
input_shape = (input_width, input_height, input_channels)

# Define the number of classes (labels)
num_classes = len(glob.glob(train_data_dir+ '/*'))

# Create the CNN model
model = create_cnn_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 298, 435, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 149, 217, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 147, 215, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 73, 107, 64)      0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 71, 105, 128)      73856     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 35, 52, 128)     

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator



# Data augmentation for training set
train_datagen = ImageDataGenerator(rescale=1.0 / 255,  # normalize pixel values to [0,1]
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   validation_split=0.2)  # 20% of the data will be used for validation

# Set batch size
batch_size = 8

# Training set generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(input_width, input_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Validation set generator
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(input_width, input_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Train the model
epochs = 20
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // batch_size,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples // batch_size,
                    epochs=epochs)

Found 575 images belonging to 23 classes.


Found 138 images belonging to 23 classes.
Epoch 1/20
71/71 [==============================] - 15s 201ms/step - loss: 3.3892 - accuracy: 0.0617 - val_loss: 2.9566 - val_accuracy: 0.1324
Epoch 2/20
71/71 [==============================] - 13s 186ms/step - loss: 2.8530 - accuracy: 0.1287 - val_loss: 2.3335 - val_accuracy: 0.2059
Epoch 3/20
71/71 [==============================] - 13s 189ms/step - loss: 2.2755 - accuracy: 0.2998 - val_loss: 1.6884 - val_accuracy: 0.3824
Epoch 4/20
71/71 [==============================] - 13s 188ms/step - loss: 1.5874 - accuracy: 0.5220 - val_loss: 1.4298 - val_accuracy: 0.5809
Epoch 5/20
71/71 [==============================] - 13s 187ms/step - loss: 0.9685 - accuracy: 0.6966 - val_loss: 0.7620 - val_accuracy: 0.7794
Epoch 6/20
71/71 [==============================] - 13s 187ms/step - loss: 0.7826 - accuracy: 0.7619 - val_loss: 0.6844 - val_accuracy: 0.7868
Epoch 7/20
71/71 [==============================] - 13s 188ms/step - loss: 0.4512 - accuracy: 0.8730

In [15]:
model.save('models/bs_amaterasu')

INFO:tensorflow:Assets written to: models/bs_amaterasu\assets


INFO:tensorflow:Assets written to: models/bs_amaterasu\assets
